# Recommender System

In [29]:
import pandas as pd
import numpy as np
import math, re, time, sys, ast, warnings

from sklearn.preprocessing import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.metrics import mean_squared_error, pairwise_distances
from sklearn.neighbors import NearestNeighbors

warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('D:/Projects/Github/games-requirements-checker/datasets/steam_dataset/steamapi/csv/steam_dataset.csv', sep=',')

In [4]:
df.describe()

,appid,ccu,meta_score,score_rank
count,3.983100e+04,39831.000000,39831.000000,35.000000
mean,8.703842e+05,148.075117,6.479451,98.828571
std,4.447675e+05,5423.287822,20.925264,0.954424
min,1.000000e+01,0.000000,0.000000,97.000000
25%,5.009300e+05,0.000000,0.000000,98.000000
50%,8.363500e+05,0.000000,0.000000,99.000000
75%,1.225375e+06,1.000000,0.000000,100.000000
max,1.832250e+06,752584.000000,97.000000,100.000000


In [9]:
df.columns

Index(['appid', 'name', 'developer', 'publisher', 'genre', 'tags', 'is_free',
       'initialprice', 'discount', 'ccu', 'required_age',
       'detailed_description', 'about_the_game', 'short_description', 'date',
       'pc_requirements_minimum', 'pc_requirements_recommended',
       'mac_requirements_minimum', 'mac_requirements_recommended',
       'linux_requirements_minimum', 'linux_requirements_recommended',
       'steamspy_tags', 'meta_score', 'score_rank', 'header_image', 'website',
       'languages'],
      dtype='object')

In [15]:
df['about_the_game'][0]

"Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role."

Get important columns

In [22]:
df_filtered = df[['name', 'developer', 'publisher', 'genre','short_description']].fillna('N/A')
df_filtered.shape

(39831, 5)

Check missing values

In [23]:
df_filtered.isnull().sum()

name                 0
developer            0
publisher            0
genre                0
short_description    0
dtype: int64

Combinine the values of important columns into a single string

In [24]:
def get_important_features(data):
    important_features = []
    for i in range(0, data.shape[0]):
        important_features.append(data['name'][i] + ' ' + data['developer'][i] + ' ' + data['publisher'][i] + ' ' +
                                    data['genre'][i] + ' ' + data['short_description'][i])
    return important_features

In [26]:
df['important_features'] = get_important_features(df_filtered)
df.head(3)

,appid,name,developer,publisher,genre,tags,is_free,initialprice,discount,ccu,...,mac_requirements_recommended,linux_requirements_minimum,linux_requirements_recommended,steamspy_tags,meta_score,score_rank,header_image,website,languages,important_features
0,10,Counter-Strike,Valve,Valve,Action,"{'Action': 5374, 'FPS': 4797, 'Multiplayer': 3...",False,999,0,14139,...,NaN,"Linux Ubuntu 12.04, Dual-core from Intel or AM...",NaN,"Action, FPS, Multiplayer, Shooter, Classic, Te...",88.0,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,0,"English, French, German, Italian, Spanish - Sp...",Counter-Strike Valve Valve Action Play the wor...
1,10090,Call of Duty: World at War,Treyarch,Activision,Action,"{'Zombies': 1108, 'World War II': 818, 'FPS': ...",False,1999,0,519,...,NaN,NaN,NaN,"Zombies, World War II, FPS, Multiplayer, Actio...",83.0,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.callofduty.com,"English, French, Italian, German, Spanish - Spain",Call of Duty: World at War Treyarch Activision...
2,10180,Call of Duty®: Modern Warfare® 2,Infinity Ward,Activision,Action,"{'Action': 1746, 'FPS': 1415, 'Multiplayer': 1...",False,1999,0,510,...,NaN,NaN,NaN,"Action, FPS, Multiplayer, Shooter, First-Perso...",86.0,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,https://www.callofduty.com/,"English, French, German, Italian, Spanish - Spain",Call of Duty®: Modern Warfare® 2 Infinity Ward...


Create the Model

In [28]:
cm = CountVectorizer().fit_transform(df['important_features'])

In [30]:
cs = cosine_similarity(cm)
print(cs)

[[1.         0.11991282 0.25685181 ... 0.20407101 0.12016032 0.26659279]
 [0.11991282 1.         0.45836777 ... 0.17963741 0.13608276 0.2117354 ]
 [0.25685181 0.45836777 1.         ... 0.5115414  0.2267126  0.40034232]
 ...
 [0.20407101 0.17963741 0.5115414  ... 1.         0.1736921  0.35232976]
 [0.12016032 0.13608276 0.2267126  ... 0.1736921  1.         0.21610154]
 [0.26659279 0.2117354  0.40034232 ... 0.35232976 0.21610154 1.        ]]


In [31]:
cs.shape

(39831, 39831)

Get the highest game score by users

In [32]:
df['name'][:3]

0                      Counter-Strike
1          Call of Duty: World at War
2    Call of Duty®: Modern Warfare® 2
Name: name, dtype: object

In [75]:
name = 'Counter-Strike'
appid = df[df['name'] == name]['appid'].values[0]
appid

10

In [76]:
scores = list(enumerate(cs[appid]))
sorted_scores = sorted(scores, key = lambda x:x[1], reverse=True)
sorted_scores = sorted_scores[1:]

In [83]:
df[df['appid'] == 730]

,appid,name,developer,publisher,genre,tags,is_free,initialprice,discount,ccu,...,mac_requirements_recommended,linux_requirements_minimum,linux_requirements_recommended,steamspy_tags,meta_score,score_rank,header_image,website,languages,important_features
885,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,"Action, Free to Play","{'FPS': 87125, 'Shooter': 62512, 'Multiplayer'...",True,0,0,752584,...,NaN,Минимальные: ОС: Ubuntu 12.04 Процессор: 64-би...,NaN,"FPS, Shooter, Multiplayer, Competitive, Action...",83.0,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://blog.counter-strike.net/,"English, Czech, Danish, Dutch, Finnish, French...","Counter-Strike: Global Offensive Valve, Hidden..."


In [79]:
df.iloc[sorted_scores[1283][0]]['name']

'Faceted Flight'

In [85]:
sorted_scores

[(32330, 0.4310054558901573),
 (1283, 0.4191297423079544),
 (20734, 0.4119429204355498),
 (9375, 0.41089070180665904),
 (7946, 0.4077954358702775),
 (7728, 0.40644850966246954),
 (13592, 0.40441523773192367),
 (15025, 0.40004734568283146),
 (2165, 0.3987333150731603),
 (17577, 0.3947710169758615),
 (27, 0.39300588695450855),
 (11707, 0.3929117166313372),
 (7, 0.39183587576350587),
 (6976, 0.3894179418882947),
 (37636, 0.3864447558701624),
 (3789, 0.3857917724530094),
 (18965, 0.38554980047802995),
 (1276, 0.3850606548386051),
 (920, 0.3842530640289477),
 (3369, 0.38330638305071263),
 (3424, 0.3829708431025351),
 (9500, 0.38295130526903515),
 (16209, 0.3829197905337418),
 (39081, 0.38168334238107365),
 (9341, 0.38095469344767463),
 (32266, 0.379532097186276),
 (18237, 0.3789266242639078),
 (34374, 0.3771682545706786),
 (658, 0.37605071654517735),
 (4086, 0.37563935738845655),
 (8977, 0.37553380809940534),
 (14607, 0.3751832396884334),
 (16427, 0.3749808688110479),
 (34082, 0.37449785079

In [48]:
len(sorted_scores)

39830

## Nearest Neighbors
Collaborative Filtering Method

In [1]:
import pandas as pd
import numpy as np
import math, re, time, sys, ast, warnings

from sklearn.metrics import mean_squared_error, pairwise_distances
from sklearn.neighbors import NearestNeighbors

In [2]:
df = pd.read_csv('D:/Projects/Github/games-requirements-checker/datasets/steam_dataset/steamapi/csv/steam_dataset.csv', sep=',')
df.head(2)

C:\Users\cilac\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (7,8,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,appid,name,developer,publisher,genre,tags,is_free,initialprice,discount,ccu,...,mac_requirements_minimum,mac_requirements_recommended,linux_requirements_minimum,linux_requirements_recommended,steamspy_tags,meta_score,score_rank,header_image,website,languages
0,10,Counter-Strike,Valve,Valve,Action,"{'Action': 5374, 'FPS': 4797, 'Multiplayer': 3...",False,999,0,14139,...,"OS X Snow Leopard 10.6.3, 1GB RAM, 4GB Hard D...",NaN,"Linux Ubuntu 12.04, Dual-core from Intel or AM...",NaN,"Action, FPS, Multiplayer, Shooter, Classic, Te...",88.0,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,0,"English, French, German, Italian, Spanish - Sp..."
1,10090,Call of Duty: World at War,Treyarch,Activision,Action,"{'Zombies': 1108, 'World War II': 818, 'FPS': ...",False,1999,0,519,...,NaN,NaN,NaN,NaN,"Zombies, World War II, FPS, Multiplayer, Actio...",83.0,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.callofduty.com,"English, French, Italian, German, Spanish - Spain"


In [3]:
df.columns

Index(['appid', 'name', 'developer', 'publisher', 'genre', 'tags', 'is_free',
       'initialprice', 'discount', 'ccu', 'required_age',
       'detailed_description', 'about_the_game', 'short_description', 'date',
       'pc_requirements_minimum', 'pc_requirements_recommended',
       'mac_requirements_minimum', 'mac_requirements_recommended',
       'linux_requirements_minimum', 'linux_requirements_recommended',
       'steamspy_tags', 'meta_score', 'score_rank', 'header_image', 'website',
       'languages'],
      dtype='object')